In [ ]:
%load_ext watermark


In [ ]:
import os

import covasim as cv
import numpy as np
from teeplot import teeplot as tp

import pylib  # noqa: F401


In [ ]:
%watermark -diwmuv -iv


In [ ]:
teeplot_subdir = os.environ.get("2025-03-31-dev-variantsync", None)
teeplot_subdir


## Quickstart

Two Strains, with lower-transmissibility variants


In [ ]:
# from https://github.com/InstituteforDiseaseModeling/covasim/blob/fef033fed5a0ca71be9c4f773d75dce1d237e753/covasim/parameters.py
gamma = dict(
    rel_beta=2.05,  # Estimated to be 1.7–2.4-fold more transmissible than wild-type: https://science.sciencemag.org/content/early/2021/04/13/science.abh2644
    rel_symp_prob=1.0,
    rel_severe_prob=2.6,  # From https://www.eurosurveillance.org/content/10.2807/1560-7917.ES.2021.26.16.2100348
    rel_crit_prob=1.0,
    rel_death_prob=1.0,
)
gamma_ = {**gamma, "rel_beta": 1.5}  # lower-transmissibility variant

delta = dict(
    rel_beta=2.2,  # Estimated to be 1.25-1.6-fold more transmissible than B117: https://www.researchsquare.com/article/rs-637724/v1
    rel_symp_prob=1.0,
    rel_severe_prob=3.2,  # 2x more transmissible than alpha from https://mobile.twitter.com/dgurdasani1/status/1403293582279294983
    rel_crit_prob=1.0,
    rel_death_prob=1.0,
)
delta_ = {**delta, "rel_beta": 1.6}  # lower-transmissibility variant


In [ ]:
# import gamma and then delta 30 days later
gamma = cv.variant(variant=gamma, label="gamma", days=0, n_imports=10)
delta = cv.variant(variant=delta, label="delta", days=30, n_imports=100)

# lower-transimissibility variants arise from spontaneous mutations, not imports
gamma_ = cv.variant(variant=gamma_, label="gamma'", days=0, n_imports=0)
delta_ = cv.variant(variant=delta_, label="delta'", days=0, n_imports=0)


In [ ]:
sim = cv.Sim(
    pop_infected=0,  # disable wild-type strain
    variants=[gamma, gamma_, delta, delta_],
    rand_seed=1,
)
sim.run()
tp.tee(
    sim.plot,
    "variant",
    teeplot_outattrs={"what": "quickstart"},
)
...


## Spontaneously Transform to Lower-Transmissibility Variants


In [ ]:
pop_size = 10_000
num_variants = 5  # 0th variant is wild-type


In [ ]:
def introduce_low_transmissibility_variants(sim: cv.Sim) -> None:
    people = sim.people

    random_mask = (
        np.random.rand(
            *people["infectious_variant"].shape,
        )
        < 0.01
    )

    has_gamma = people["infectious_variant"] == 1
    people["infectious_variant"][has_gamma & random_mask] = 2

    has_delta = people["infectious_variant"] == 3
    people["infectious_variant"][has_delta & random_mask] = 4


In [ ]:
sim = cv.Sim(
    interventions=[introduce_low_transmissibility_variants],
    pop_infected=0,  # disable wild-type strain
    pop_size=pop_size,
    variants=[gamma, gamma_, delta, delta_],
    rand_seed=1,
)
sim.run()
tp.tee(
    sim.plot,
    "variant",
    teeplot_outattrs={"what": "spontaneous-transform"},
)
...
